# 📐 Documentación de Fórmulas y Feature Engineering

## Sistema de Ruteo Seguro para CDMX - Análisis de Accidentes de Tránsito

Este documento describe **todas las fórmulas matemáticas, ponderaciones y columnas** utilizadas en el proyecto, desde el procesamiento inicial hasta el sistema de ruteo final.

---

## 📊 Tabla de Contenido

1. [Columnas del Dataset Original](#1.-Columnas-del-Dataset-Original)
2. [Feature Engineering - Columnas Generadas](#2.-Feature-Engineering---Columnas-Generadas)
3. [Fórmulas de Análisis Espacial](#3.-Fórmulas-de-Análisis-Espacial)
4. [Fórmulas de Machine Learning](#4.-Fórmulas-de-Machine-Learning)
5. [Fórmulas de Ruteo y Ponderación de Calles](#5.-Fórmulas-de-Ruteo-y-Ponderación-de-Calles)
6. [Índices Compuestos y Scoring Final](#6.-Índices-Compuestos-y-Scoring-Final)

---

## 1. Columnas del Dataset Original

### 📂 Datos de Accidentes C5 CDMX (2019-2023)

**Columnas originales del dataset de incidentes viales:**

| Columna | Tipo | Descripción | Origen |
|---------|------|-------------|--------|
| `id` | String | Identificador único del accidente | Original C5 |
| `edo` | String | Estado (Ciudad de México) | Original C5 |
| `mes` | Integer | Mes del accidente (1-12) | Original C5 |
| `anio` | Integer | Año del accidente | Original C5 |
| `mpio` | String | Municipio/Alcaldía | Original C5 |
| `hora` | Integer | Hora del día (0-23) | Original C5 |
| `minutos` | Integer | Minutos (0-59) | Original C5 |
| `dia` | Integer | Día del mes | Original C5 |
| `diasemana` | Integer | Día de la semana (0=Lunes, 6=Domingo) | Original C5 |
| `urbana` | Integer | ¿Es zona urbana? (1=Sí, 0=No) | Original C5 |
| `latitud` | Float | Coordenada latitud WGS84 | Original C5 |
| `longitud` | Float | Coordenada longitud WGS84 | Original C5 |
| `CLASACC` | String | Clasificación del accidente (Fatal/Solo Daños/etc.) | Original C5 |
| `tipo_accidente` | String | Tipo específico (Colisión/Atropello/etc.) | Original C5 |
| `delegacion` | String | Alcaldía donde ocurrió | Original C5 |

**Total de registros:** 32,139 accidentes (2019-2023)

---

## 2. Feature Engineering - Columnas Generadas

### 🔧 Procesamiento en `proceso.ipynb`

#### 2.1 Matching con Red Vial OSM

**Columnas agregadas al vincular accidentes con la red vial:**

| Columna | Fórmula/Método | Descripción |
|---------|----------------|-------------|
| `nodo_cercano` | $\text{argmin}(\sqrt{(x_{\text{nodo}} - \text{lon})^2 + (y_{\text{nodo}} - \text{lat})^2})$ | Nodo OSM más cercano al accidente |
| `distancia_nodo` | $\sqrt{(x_{\text{nodo}} - \text{lon})^2 + (y_{\text{nodo}} - \text{lat})^2} \times 111{,}000 \text{ m}$ | Distancia euclidiana en metros |
| `u`, `v` | Topología de grafo | Nodos de inicio/fin del tramo donde ocurrió |
| `tramo_id` | `hash(u, v, key)` | Identificador único del segmento vial |

**Fórmula de distancia euclidiana en coordenadas geográficas:**

$$
d(\text{m}) = \sqrt{(\text{lon}_2 - \text{lon}_1)^2 + (\text{lat}_2 - \text{lat}_1)^2} \times 111{,}000
$$

Donde 111 km ≈ 1° de latitud a nivel del ecuador.

#### 2.2 Agregación por Tramo Vial

**Estadísticas calculadas por cada tramo (u, v):**

| Métrica | Fórmula | Descripción |
|---------|---------|-------------|
| `total_accidentes` | $\text{COUNT}(*)$ | Número total de accidentes en el tramo |
| `accidentes_fatales` | $\sum (\text{CLASACC} == \text{'Fatal'})$ | Accidentes con víctimas mortales |
| `accidentes_graves` | $\sum (\text{CLASACC} == \text{'Grave'})$ | Accidentes con lesionados graves |
| `tasa_gravedad` | $\frac{\text{fatales} + \text{graves}}{\text{total}}$ | Proporción de accidentes severos |
| `latitud_centro` | $\text{MEAN}(\text{latitud})$ | Centroide del tramo |
| `longitud_centro` | $\text{MEAN}(\text{longitud})$ | Centroide del tramo |

**Salida:** `STATS_POR_TRAMO_2019_2023.csv` - 14,982 tramos analizados

### 🔧 Procesamiento en `01_analisis_espacial_clustering.ipynb`

#### 2.3 Clustering Espacial (DBSCAN)

**Columnas generadas:**

| Columna | Algoritmo | Descripción |
|---------|-----------|-------------|
| `cluster_dbscan` | DBSCAN(eps=200m, min_samples=20) | ID del cluster (-1 = ruido) |
| `riesgo_cluster` | Ver fórmula abajo | Índice de riesgo basado en clustering |

**Fórmula de riesgo por clustering:**

$$
\text{riesgo\_cluster} = \begin{cases}
50 & \text{si cluster\_id} \neq -1 \\
0 & \text{si cluster\_id} = -1 \text{ (ruido)}
\end{cases}
$$

$$
\text{cluster\_sizes} = \text{COUNT}(*) \; \text{GROUP BY cluster\_id}
$$

$$
\text{riesgo\_cluster\_ajustado} = 50 + 50 \times \frac{\text{size} - \text{size}_{\min}}{\text{size}_{\max} - \text{size}_{\min}}
$$

**Interpretación:**
- Accidentes aislados (ruido): riesgo = 0
- Cluster pequeño: riesgo ≈ 50
- Cluster más grande: riesgo → 100

**Parámetros DBSCAN:**
- `eps = 200 metros` - Radio de vecindad
- `min_samples = 20` - Mínimo de accidentes para formar cluster
- Resultado: **299 clusters identificados**, 46.6% ruido

#### 2.4 Hot Spots Analysis (Getis-Ord Gi*)

**Metodología:** Análisis de cuadrícula espacial

```python
# Crear grid de 0.01° (~1.1 km por celda)
cell_size = 0.01°
grid_cells = [box(minx + i×Δ, miny + j×Δ, ...) for i,j in product(cols, rows)]
```

**Fórmula Getis-Ord Gi*:**

$$
G_i^* = \frac{\sum_j w_{ij}x_j - \bar{X} \sum_j w_{ij}}{S \sqrt{\frac{n\sum_j w_{ij}^2 - (\sum_j w_{ij})^2}{n - 1}}}
$$

Donde:
- $x_j$ = número de accidentes en celda j
- $w_{ij}$ = 1 si dist(i,j) < 0.02°, 0 en otro caso
- $\bar{X}$ = media global de accidentes
- $S$ = desviación estándar
- $n$ = número total de celdas

**Columnas generadas:**

| Columna | Descripción | Interpretación |
|---------|-------------|----------------|
| `gi_score` | Z-score de Gi* | >2.58 = Hot Spot 99% conf. |
| `gi_pvalue` | p-value normal | <0.01 = Significativo |
| `hot_spot` | Clasificación | 'Hot Spot (99%)', 'Cold Spot', etc. |
| `riesgo_hotspot` | $50 + 50 \times \frac{\text{gi\_score}}{\max(\text{gi\_score})}$ | Índice de riesgo [0-100] |

**Salida:** `GRID_HOTSPOTS.csv` - 755 celdas con accidentes

#### 2.5 Autocorrelación Espacial (Moran's I)

**Fórmula de Moran's I Global:**

$$
I = \frac{n}{\sum_i \sum_j w_{ij}} \times \frac{\sum_i \sum_j w_{ij}(x_i - \bar{x})(x_j - \bar{x})}{\sum_i (x_i - \bar{x})^2}
$$

**Resultado obtenido:**
- $I = 0.6837$ (p < 0.001)
- Interpretación: **Clustering espacial altamente significativo**
- Los accidentes NO están distribuidos aleatoriamente

### 🔧 Procesamiento en `02_modelado_ml_causas.ipynb`

#### 2.6 Features Temporales

| Feature | Fórmula | Valores posibles |
|---------|---------|------------------|
| `hora` | Extraído de timestamp | 0-23 |
| `dia_semana` | `datetime.dayofweek` | 0-6 (0=Lunes) |
| `mes` | `datetime.month` | 1-12 |
| `es_fin_semana` | $\begin{cases}1 & \text{si dia\_semana} \geq 5 \\ 0 & \text{en otro caso}\end{cases}$ | 0, 1 |
| `periodo_dia` | Ver función abajo | 'madrugada', 'mañana', 'tarde', 'noche' |

**Función de clasificación de periodo:**

```python
def clasificar_periodo(hora):
    if 6 <= hora < 12:  return 'manana'
    if 12 <= hora < 18: return 'tarde'
    if 18 <= hora < 24: return 'noche'
    else:               return 'madrugada'  # 0-5
```

#### 2.7 Features Espaciales

| Feature | Fórmula | Descripción |
|---------|---------|-------------|
| `en_cluster` | $\begin{cases}1 & \text{si cluster\_dbscan} \neq -1 \\ 0 & \text{en otro caso}\end{cases}$ | Binario: ¿en punto negro? |
| `riesgo_cluster` | Heredado de 01_ | Índice [0-100] |

#### 2.8 Variable Objetivo

**Target binario de gravedad:**

```python
def binarizar_gravedad(CLASACC):
    if CLASACC in ['Fatal', 'Grave', 'Con lesionados']:
        return 1  # Accidente grave
    elif CLASACC in ['Solo daños']:
        return 0  # Accidente leve
    else:
        return 0
```

**Distribución:** ~15% accidentes graves (desbalanceado)

---

## 3. Fórmulas de Análisis Espacial

### 3.1 DBSCAN - Density-Based Spatial Clustering

**Algoritmo:**

1. Para cada punto p:
   - $N(p) = \{q : \text{dist}(p,q) \leq \varepsilon\}$
   - Si $|N(p)| \geq \text{min\_samples}$ → p es **core point**

2. Conectividad:
   - p y q están en mismo cluster si existe camino de core points

**Distancia en coordenadas proyectadas (UTM):**

$$
\text{dist}(p, q) = \sqrt{(x_p - x_q)^2 + (y_p - y_q)^2}
$$

Se usa EPSG:32614 (UTM Zone 14N) para CDMX.

### 3.2 Matriz de Pesos Espaciales (W)

**Distance Band Weights:**

$$
w_{ij} = \begin{cases}
1 & \text{si dist}(i,j) \leq \text{threshold} \\
0 & \text{en otro caso}
\end{cases}
$$

- `threshold = 0.02°` (~2.2 km)
- Matriz binaria (sin ponderación por distancia)

### 3.3 Getis-Ord Gi* (Detección de Hot Spots)

**Versión estandarizada (Z-score):**

$$
G_i^* = \frac{\sum_j w_{ij}x_j - \bar{X} \sum_j w_{ij}}{S \sqrt{\frac{n\sum w_{ij}^2 - (\sum w_{ij})^2}{n-1}}}
$$

**Interpretación del Z-score:**

| Gi* (Z-score) | p-value | Clasificación |
|---------------|---------|---------------|
| > 2.58 | < 0.01 | Hot Spot (99% confianza) |
| > 1.96 | < 0.05 | Hot Spot (95% confianza) |
| -1.96 a 1.96 | > 0.05 | No significativo |
| < -1.96 | < 0.05 | Cold Spot (95% confianza) |
| < -2.58 | < 0.01 | Cold Spot (99% confianza) |

- **Valores positivos** → Clustering de valores altos (muchos accidentes)
- **Valores negativos** → Clustering de valores bajos (pocos accidentes)

### 3.4 Moran's I (Autocorrelación Espacial)

**Valores de I:**
- $I = -1$ → Dispersión perfecta (patrón de tablero de ajedrez)
- $I = 0$ → Distribución aleatoria (sin autocorrelación)
- $I = +1$ → Clustering perfecto (valores similares juntos)

**Nuestro resultado: I = 0.6837**
- Fuerte clustering espacial
- Los accidentes tienden a ocurrir cerca de otros accidentes
- Justifica el uso de análisis espacial

---

## 4. Fórmulas de Machine Learning

### 4.1 Modelos Entrenados

#### Decision Tree

$$
\text{Gini Impurity} = 1 - \sum_i p_i^2
$$

$$
\text{Information Gain} = \text{Gini}(\text{parent}) - \sum \frac{n_{\text{child}}}{n_{\text{parent}}} \times \text{Gini}(\text{child})
$$

**Hiperparámetros:**
- `max_depth = 5` - Profundidad máxima del árbol
- `min_samples_split = 100` - Mínimo para dividir nodo
- `min_samples_leaf = 50` - Mínimo en hoja

#### Random Forest

**Ensemble de N árboles de decisión:**

$$
P_{\text{ensemble}}(\text{grave} | X) = \frac{1}{N} \sum_{i=1}^{N} P_i(\text{grave} | X)
$$

**Hiperparámetros:**
- `n_estimators = 100` - Número de árboles
- `max_depth = 10`
- `min_samples_split = 50`
- `min_samples_leaf = 20`

**Feature Importance:**

$$
\text{importance}(f_j) = \frac{\sum_t (\text{impurity\_decrease} \times n_{\text{samples}})}{\text{total\_samples}}
$$

Promediado sobre todos los árboles del bosque.

#### Logistic Regression

**Función logística:**

$$
P(y=1 | X) = \frac{1}{1 + e^{-z}}
$$

donde:

$$
z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_n x_n
$$

**Optimización:** Maximum Likelihood Estimation (MLE)

#### Stacking Ensemble

**Arquitectura de 2 niveles:**

```
Level 0 (Base learners):
  - Decision Tree
  - Random Forest
  - Logistic Regression

Level 1 (Meta-learner):
  - Logistic Regression entrenada sobre predicciones de Level 0
```

**Predicción final:**

$$
P_{\text{stack}}(\text{grave} | X) = \sigma(w_1 P_{DT} + w_2 P_{RF} + w_3 P_{LR} + b)
$$

Donde $\sigma()$ es la función logística y $w_1, w_2, w_3, b$ son parámetros aprendidos.

### 4.2 Métricas de Evaluación

**Accuracy:**
$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

**Precision y Recall:**
$$
\text{Precision} = \frac{TP}{TP + FP}
$$

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

**F1-Score:**
$$
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

**AUC-ROC:**
- Área bajo la curva ROC
- Mide capacidad de discriminación del modelo
- Valores: 0.5 (random) a 1.0 (perfecto)

---

## 5. Fórmulas de Ruteo y Ponderación de Calles

### 5.1 Asignación de Riesgo a Aristas (Calles)

**Para cada arista (u, v) del grafo:**

```python
# Paso 1: Matching espacial con estadísticas de tramo
lat_centro = (lat_u + lat_v) / 2
lon_centro = (lon_u + lon_v) / 2

tramos_cercanos = df_stats[
    (|lat_stats - lat_centro| < 0.005) &
    (|lon_stats - lon_centro| < 0.005)
]

# Paso 2: Normalización del riesgo
if len(tramos_cercanos) > 0:
    accidentes = MEAN(tramos_cercanos.total_accidentes)
    riesgo = (accidentes / max_accidentes) × 100
else:
    riesgo = 25  # Valor por defecto
```

**Interpretación:**
- `riesgo = 0` → Sin accidentes registrados
- `riesgo = 50` → Promedio
- `riesgo = 100` → Calle con más accidentes en CDMX

### 5.2 Pesos para Ruteo (Costos de Aristas)

**Se definen 3 funciones de peso diferentes:**

#### Peso 1: Solo Distancia (Ruta Más Corta)

$$
\text{peso\_distancia} = \text{longitud} \quad [\text{metros}]
$$

- Minimiza distancia total
- Ignora completamente el riesgo
- Algoritmo: Dijkstra con peso = longitud

#### Peso 2: Balanceado (Trade-off)

$$
\text{peso\_balanceado} = \text{longitud} \times \left(1 + \frac{\text{riesgo}}{100}\right)
$$

**Ejemplo:**
- Calle de 1000m con riesgo 0: peso = 1000 × 1.0 = 1000
- Calle de 1000m con riesgo 50: peso = 1000 × 1.5 = 1500
- Calle de 1000m con riesgo 100: peso = 1000 × 2.0 = 2000

**Interpretación:** Una calle con riesgo=100 se "siente" 2× más larga.

#### Peso 3: Seguro (Prioriza Seguridad)

$$
\text{peso\_seguro} = \text{longitud} \times \left(1 + \frac{2 \times \text{riesgo}}{100}\right)
$$

**Ejemplo:**
- Calle de 1000m con riesgo 0: peso = 1000 × 1.0 = 1000
- Calle de 1000m con riesgo 50: peso = 1000 × 2.0 = 2000
- Calle de 1000m con riesgo 100: peso = 1000 × 3.0 = 3000

**Interpretación:** Una calle con riesgo=100 se "siente" 3× más larga.
El factor 2 amplifica la penalización por riesgo.

### 5.3 Algoritmo de Ruteo (Dijkstra)

**Para encontrar camino mínimo de origen a destino:**

```
1. Inicializar:
   dist[origen] = 0
   dist[v] = ∞ para v ≠ origen
   Q = todos los nodos

2. Mientras Q no esté vacío:
   u = nodo en Q con dist[u] mínima
   Remover u de Q

   Para cada vecino v de u:
       alt = dist[u] + peso(u, v)
       if alt < dist[v]:
           dist[v] = alt
           prev[v] = u

3. Reconstruir ruta desde destino hacia origen usando prev[]
```

**Complejidad:** $O((V + E) \log V)$ con heap binario

### 5.4 Métricas de Rutas Calculadas

Para cada ruta $[\text{nodo}_1, \text{nodo}_2, \ldots, \text{nodo}_n]$:

$$
\text{distancia\_total} = \sum_i \text{longitud}(\text{nodo}_i, \text{nodo}_{i+1})
$$

$$
\text{riesgo\_promedio} = \frac{1}{n} \sum_i \text{riesgo}(\text{nodo}_i, \text{nodo}_{i+1})
$$

$$
\text{riesgo\_maximo} = \max\{\text{riesgo}(\text{nodo}_i, \text{nodo}_{i+1})\}
$$

$$
\text{riesgo\_total} = \sum_i \text{riesgo}(\text{nodo}_i, \text{nodo}_{i+1})
$$

**Score de Seguridad:**
$$
\text{score\_seguridad} = 100 - \text{riesgo\_promedio}
$$

Valores altos → ruta más segura

---

## 6. Índices Compuestos y Scoring Final

### 6.1 Riesgo ML (Machine Learning)

**Predicción de probabilidad de gravedad:**

```python
# Modelo Random Forest entrenado
riesgo_ml = P(grave | features) × 100
```

Donde `features = [hora, dia_semana, mes, es_fin_semana, en_cluster, riesgo_cluster, ...]`

**Interpretación:**
- `riesgo_ml = 10` → 10% probabilidad de accidente grave
- `riesgo_ml = 50` → 50% probabilidad
- `riesgo_ml = 90` → 90% probabilidad

### 6.2 Índice de Riesgo Compuesto FINAL

**Fórmula de combinación (3 capas de riesgo):**

$$
\text{riesgo\_compuesto} = 0.6 \times \text{riesgo\_histórico} + 0.1 \times \text{riesgo\_cluster} + 0.3 \times \text{riesgo\_ml}
$$

**Componentes del índice:**

1. **Riesgo Histórico (60%):** Basado en accidentes reales por tramo
   - Fuente: `STATS_POR_TRAMO_2019_2023.csv`
   - Cálculo: `(accidentes_tramo / max_accidentes) × 100`
   - Matching: Por IDs exactos (edge_u, edge_v, edge_key)

2. **Riesgo Clustering (10%):** Basado en densidad espacial (DBSCAN + Getis-Ord)
   - Fuente: `ACCIDENTES_CON_CLUSTERING.csv` + `GRID_HOTSPOTS.csv`
   - Cálculo: `50 + 50 × (tamaño_cluster - min) / (max - min)`
   - Matching: Spatial matching con KDTree (<500m)

3. **Riesgo ML (30%):** Predicción de gravedad con Random Forest
   - Fuente: `SCORING_RIESGO_COMPUESTO.csv`
   - Cálculo: `P(grave | features) × 100`
   - Matching: Spatial matching con KDTree (<500m)

**Justificación de pesos:**
- **60% histórico:** Los datos reales de accidentes son el indicador más confiable y directo
- **30% ML:** La predicción de gravedad añade contexto importante (hora, día, condiciones)
- **10% clustering:** Complementa identificando patrones espaciales de concentración

**Rango:** [0, 100]
- 0 = Riesgo mínimo (sin accidentes históricos, sin clustering, baja probabilidad ML)
- 100 = Riesgo máximo (máximos accidentes, cluster grande, alta probabilidad ML)

**Implementación en `03_sistema_ruteo_zocalo_polanco.ipynb`:**

```python
def asignar_riesgo_a_edges_completo(G, df_stats, df_scoring, df_clusters):
    # 1. Riesgo Histórico (matching exacto por IDs)
    lookup_key = (str(edge_u), str(edge_v), edge_key)
    riesgo_historico = stats_dict.get(lookup_key, 25)  # default: medio
    
    # 2. Riesgo Clustering + ML (búsqueda espacial)
    dist, idx = tree.query([lat_centro, lon_centro], k=1)
    if dist < 0.005:  # <500m
        riesgo_clustering = df_scoring.iloc[idx]['riesgo_cluster']
        riesgo_ml = df_scoring.iloc[idx]['riesgo_ml']
    else:
        riesgo_clustering = 0   # Sin cluster detectado
        riesgo_ml = 15          # Riesgo bajo-medio por defecto
    
    # 3. Combinar con ponderación
    riesgo_compuesto = (
        0.6 * riesgo_historico +
        0.1 * riesgo_clustering +
        0.3 * riesgo_ml
    )
    
    return riesgo_compuesto
```

### 6.3 Fórmula Final de Comparación de Rutas

**Para recomendar ruta óptima:**

```python
# Trade-off normalizado
diff_distancia_pct = (dist_segura - dist_corta) / dist_corta × 100
diff_riesgo_pct = (riesgo_corta - riesgo_segura) / riesgo_corta × 100

if diff_riesgo_pct > 20 and diff_distancia_pct < 25:
    recomendacion = "Ruta Más Segura"
elif diff_riesgo_pct > 10:
    recomendacion = "Ruta Balanceada"
else:
    recomendacion = "Ruta Más Corta"
```

**Criterio de decisión:**
- Si la ruta segura reduce riesgo >20% y aumenta distancia <25% → Vale la pena
- Si reduce riesgo 10-20% → Balancear
- Si reduce riesgo <10% → Priorizar eficiencia

### 6.4 Normalización Multidimensional (Radar Chart)

**Para comparar rutas en 3 dimensiones:**

```python
# Seguridad (invertir riesgo)
seguridad_score = 100 - riesgo_promedio

# Eficiencia (normalizar distancia)
dist_min = min(distancias_todas_rutas)
eficiencia_score = 100 - ((distancia - dist_min) / dist_min × 50)

# Estabilidad (evitar picos de riesgo)
estabilidad_score = 100 - riesgo_maximo
```

**Área del polígono en radar chart:**
- Mayor área → Mejor desempeño general
- Forma del polígono muestra fortalezas/debilidades

---

## 📌 Resumen de Columnas por Archivo

### ACCIDENTES_CON_TRAMOS_2019_2023.csv
- Columnas originales C5 (15)
- `+ nodo_cercano, u, v, tramo_id` (4)
- **Total: 19 columnas**

### STATS_POR_TRAMO_2019_2023.csv
- `u, v, total_accidentes, accidentes_fatales, accidentes_graves`
- `tasa_gravedad, latitud_centro, longitud_centro`
- **Total: 8 columnas, 14,982 tramos**

### ACCIDENTES_CON_CLUSTERING.csv
- `latitud, longitud, cluster_dbscan, riesgo_cluster`
- **Total: 4 columnas, 32,139 registros**

### GRID_HOTSPOTS.csv
- `centroid_lat, centroid_lon, n_accidentes`
- `gi_score, hot_spot, riesgo_hotspot`
- **Total: 6 columnas, 755 celdas**

### SCORING_RIESGO_COMPUESTO.csv
- `riesgo_cluster, riesgo_ml, indice_riesgo_compuesto`
- `latitud, longitud` (opcional)
- **Total: 5 columnas**

---

## 🔬 Validación Científica

### Significancia Estadística

1. **Moran's I:** p < 0.001 ✓
   - Clustering espacial altamente significativo

2. **Getis-Ord Gi*:** Z > 2.58 para hot spots
   - Confianza del 99% en zonas identificadas

3. **Modelos ML:** AUC > 0.70
   - Capacidad predictiva superior al azar

### Supuestos y Limitaciones

1. **Distancia euclidiana:** Aproximación válida para áreas pequeñas (<10 km)
2. **Estacionariedad:** Asumimos que patrones son estables 2019-2023
3. **Completitud de datos:** No todos los accidentes son reportados
4. **Red vial OSM:** Puede tener calles faltantes o desactualizadas

---

## 📚 Referencias Matemáticas

- **DBSCAN:** Ester et al. (1996) "A density-based algorithm for discovering clusters"
- **Getis-Ord Gi*:** Ord & Getis (1995) "Local Spatial Autocorrelation Statistics"
- **Moran's I:** Moran (1950) "Notes on continuous stochastic phenomena"
- **Random Forest:** Breiman (2001) "Random Forests"
- **Dijkstra:** Dijkstra (1959) "A note on two problems in connexion with graphs"

---

## 🎯 Aplicación Práctica

**Ejemplo: Zócalo → Polanco**

```
Ruta Más Corta:
  - Distancia: 8.5 km
  - Riesgo promedio: 45/100
  - Fórmula: argmin ∑ longitud(e)

Ruta Balanceada:
  - Distancia: 9.2 km (+8%)
  - Riesgo promedio: 32/100 (-29%)
  - Fórmula: argmin ∑ longitud(e) × (1 + riesgo/100)

Ruta Más Segura:
  - Distancia: 10.1 km (+19%)
  - Riesgo promedio: 25/100 (-44%)
  - Fórmula: argmin ∑ longitud(e) × (1 + 2×riesgo/100)

Recomendación: Ruta Más Segura
  - Reduce riesgo 44% con solo 19% más distancia
  - Tiempo adicional: ~7 minutos
```

---

**Última actualización:** 2025-11-20

**Versión:** 1.0

**Autores:** Equipo Proyecto Final Minería de Datos